In [1]:
%load_ext autoreload
%autoreload v2

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [13]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os

In [4]:
tf.__version__

'2.0.0-alpha0'

### Basic MNIST

In [5]:
mnist = tf.keras.datasets.mnist

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)

In [11]:
model.fit(x_train, y_train, epochs=5, callbacks=[cp_callback])
model.evaluate(x_test, y_test)

Epoch 1/5
59744/60000 [============================>.] - ETA: 0s - loss: 0.2934 - accuracy: 0.9141
Epoch 00001: saving model to training_1/cp.ckpt
60000/60000 [==============================] - 12s 194us/sample - loss: 0.2935 - accuracy: 0.9140
Epoch 2/5
59808/60000 [============================>.] - ETA: 0s - loss: 0.1443 - accuracy: 0.9569
Epoch 00002: saving model to training_1/cp.ckpt
60000/60000 [==============================] - 10s 171us/sample - loss: 0.1442 - accuracy: 0.9570
Epoch 3/5
59808/60000 [============================>.] - ETA: 0s - loss: 0.1064 - accuracy: 0.9675
Epoch 00003: saving model to training_1/cp.ckpt
60000/60000 [==============================] - 11s 184us/sample - loss: 0.1064 - accuracy: 0.9675
Epoch 4/5
59872/60000 [============================>.] - ETA: 0s - loss: 0.0885 - accuracy: 0.9730
Epoch 00004: saving model to training_1/cp.ckpt
60000/60000 [==============================] - 10s 168us/sample - loss: 0.0885 - accuracy: 0.9730
Epoch 5/5
59904/6000

[0.07973091734303162, 0.9768]

In [12]:
model.load_weights(checkpoint_path)

### Advanced MNIST

In [14]:
dataset, info = tfds.load('mnist', with_info=True, as_supervised=True)

In [15]:
def convert_types(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

In [16]:
dataset['train']

<_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

In [18]:
mnist_train = dataset['train'].map(convert_types).shuffle(10000).batch(32)
mnist_test = dataset['test'].map(convert_types).batch(32)

In [19]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(128, activation='relu')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')

    @tf.function
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

In [59]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  320       
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  2769024   
_________________________________________________________________
dense_1 (Dense)              multiple                  1290      
Total params: 2,770,634
Trainable params: 2,770,634
Non-trainable params: 0
_________________________________________________________________


In [20]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [21]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [22]:
@tf.function
def train_step(image, label):
    with tf.GradientTape() as tape:
        predictions = model(image)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(label, predictions)

    
# Pushing code to graph.
@tf.function
def test_step(image, label):
    predictions = model(image)
    t_loss = loss_object(label, predictions)

    test_loss(t_loss)
    test_accuracy(label, predictions)

In [23]:
def predict(image):
    return model(image)

In [24]:
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

In [25]:
last_loss = 999
for epoch in range(5):
    for image, label in mnist_train:
        train_step(image, label)

    for test_image, test_label in mnist_test:
        test_step(test_image, test_label)
    
    ckpt.step.assign_add(1)
    if test_loss.result() < last_loss:
        last_loss = test_loss.result()
        save_path = manager.save()
        print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1, train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))

Saved checkpoint for step 2: ./tf_ckpts/ckpt-1
Epoch 1, Loss: 0.1374327689409256, Accuracy: 95.88166809082031, Test Loss: 0.06837800145149231, Test Accuracy: 97.70999908447266
Saved checkpoint for step 3: ./tf_ckpts/ckpt-2
Epoch 2, Loss: 0.09150621294975281, Accuracy: 97.22750091552734, Test Loss: 0.060234107077121735, Test Accuracy: 98.04000091552734
Saved checkpoint for step 4: ./tf_ckpts/ckpt-3
Epoch 3, Loss: 0.06872385740280151, Accuracy: 97.90721893310547, Test Loss: 0.057813432067632675, Test Accuracy: 98.16667175292969
Saved checkpoint for step 5: ./tf_ckpts/ckpt-4
Epoch 4, Loss: 0.055405694991350174, Accuracy: 98.31541442871094, Test Loss: 0.05761345848441124, Test Accuracy: 98.22999572753906
Epoch 5, Loss: 0.046451643109321594, Accuracy: 98.58466339111328, Test Loss: 0.058373551815748215, Test Accuracy: 98.26800537109375


In [26]:
manager.latest_checkpoint

'./tf_ckpts/ckpt-4'

In [27]:
ckpt.restore(manager.latest_checkpoint)

In [33]:
data, labels = next(iter(mnist_test))
result = model(data)
np.mean(np.argmax(result.numpy(), axis=1) == labels.numpy())

0.9375

In [32]:
prediction = model(np.zeros([32, 28, 28, 1], dtype=np.float32))

### Datasets

In [35]:
train, test = tf.keras.datasets.mnist.load_data()

In [36]:
train[1].shape

(60000,)

In [37]:
mnist_ds = tf.data.Dataset.from_tensor_slices(train)
mnist_ds = mnist_ds.shuffle(1000000).batch(5)

In [38]:
next(iter(mnist_ds))

(<tf.Tensor: id=102606, shape=(5, 28, 28), dtype=uint8, numpy=
 array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, 

In [53]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

In [41]:
maybe_download()

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 1us/step


('/home/adam/.keras/datasets/iris_training.csv',
 '/home/adam/.keras/datasets/iris_test.csv')

In [42]:
irs_ds = tf.data.TextLineDataset('/home/adam/.keras/datasets/iris_training.csv').skip(1)

In [45]:
irs_ds = irs_ds.shuffle(100).batch(5)

In [46]:
next(iter(irs_ds))

<tf.Tensor: id=23831, shape=(5,), dtype=string, numpy=
array([b'5.4,3.7,1.5,0.2,0', b'6.3,3.4,5.6,2.4,2', b'4.9,3.0,1.4,0.2,0',
       b'6.4,2.7,5.3,1.9,2', b'6.0,2.9,4.5,1.5,1'], dtype=object)>

In [56]:
CSV_TYPES = [[0.0], [0.0], [0.0], [0.0], [0]]
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']


def _parse_line(line):
    # Decode the line into its fields
    fields = tf.io.decode_csv(line, record_defaults=CSV_TYPES)

    # Pack the result into a dictionary
    features = dict(zip(CSV_COLUMN_NAMES, fields))

    # Separate the label from the features
    label = features.pop('Species')

    return features, label

In [57]:
irs_ds = tf.data.TextLineDataset('/home/adam/.keras/datasets/iris_training.csv').skip(1)
irs_ds = irs_ds.map(_parse_line).shuffle(100).batch(5)

In [58]:
next(iter(irs_ds))

({'PetalLength': <tf.Tensor: id=23886, shape=(5,), dtype=float32, numpy=array([4.5, 4.8, 4.8, 4.9, 1.3], dtype=float32)>,
  'PetalWidth': <tf.Tensor: id=23887, shape=(5,), dtype=float32, numpy=array([1.5, 1.8, 1.8, 1.5, 0.3], dtype=float32)>,
  'SepalLength': <tf.Tensor: id=23888, shape=(5,), dtype=float32, numpy=array([6.4, 6. , 5.9, 6.9, 5. ], dtype=float32)>,
  'SepalWidth': <tf.Tensor: id=23889, shape=(5,), dtype=float32, numpy=array([3.2, 3. , 3.2, 3.1, 3.5], dtype=float32)>},
 <tf.Tensor: id=23890, shape=(5,), dtype=int32, numpy=array([1, 2, 1, 1, 0], dtype=int32)>)